In [46]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
import requests
from bs4 import CData
from bs4 import BeautifulSoup
res = []
pages_num = 3

for i in range(1, pages_num):
    url = "https://www.avito.ru/moskva/kvartiry/sdam/na_dlitelnyy_srok?p=" + str(i) +"&pmax=40000&pmin=0&metro=7-25-30-110&f=550_5702-5703-5704"

    session = requests.Session()
    html_code = session.get(url)
    soup = BeautifulSoup(html_code.text, 'html.parser')

    for elem in soup.find_all("div", {"class": "item_table"}):
        res.append([])
        for d in elem.find_all("div", {"class": "description"}):
            s = d.find("div", {"class": "about"}).text.strip().split('\n')[0].strip()
            s = s.replace(" руб. в месяц", "")
            s = s.replace(" ", "")
            res[-1].append(s)
            v = d.find("div", {"class": "item_table-header"}).h3.text.strip().split(", ")
            v[0] = v[0][0]
            v[1] = v[1].replace(" м²", "")
            v[2] = v[2].replace(" эт.", "")
            res[-1] += v
            s = d.find("i", {"class": "i-metro"}).next_element.string.strip()
            res[-1].append(s)
            s = d.find("i", {"class": "i-metro"}).next_element.next_element.string.strip()
            if (s.find("км")):
                s = s.replace(" км", "00")
                if (not(s.find("."))):
                    s += "0"
                else:
                    s = s.replace(".", "")
            if (s.find(" м")):
                s = s.replace(" м", "")
            res[-1].append(s)
            s = d.find("i", {"class": "i-metro"}).next_element.next_element.next_element.next_element.string.strip()
            s = s[2:]
            for i in range (0,len(s)):
                if (ord(s[i]) == 160):
                    s = s.replace(s[i], " ")
            res[-1].append(s)
f = open('avito.txt', 'w')
for i in range (0, len(res)):
    for j in range (0, len(res[i])):
        f.write(str(res[i][j]).ljust(15))
    f.write('\n')
f.close()

In [47]:
import requests
from bs4 import CData
from bs4 import BeautifulSoup

res = []
pages_num = 7
for i in range(1, pages_num):
    url = "https://www.domofond.ru/arenda-nedvizhimosti/search?MetroIds=21%2C44%2C124%2C39&PropertyTypeDescription=kvartiry&PriceTo=40000&RentalRate=Month&Rooms=One%2CTwo&Page="+str(i)
    session = requests.Session()
    html_code = session.get(url)
    soup = BeautifulSoup(html_code.text, 'html.parser')

    for elem in soup.find_all("div", {"class": "media-body e-content"}):
        res.append([])
        s = elem.a.h2.text
        for i in range (0, len(s)):
            if (ord(s[i]) == 160):
                s = s.replace(s[i], " ")
        s = s.replace(" ", "")
        s = s.replace("РУБ.", "")
        res[-1].append(s)
        for d in elem.find_all("span", {"class": "e-tile-type m-max-width text-overflow"}):
            v = d.text.strip().split(", ")
            v[0] = v[0][0]
            v[1] = v[1].replace(" м²", "")
            v[2] = v[2].replace(" эт.", "")
            res[-1] += (v)
        mtr = elem.find("div", {"class": "e-metro-distance"})
        mtr = mtr.strong.text.split(" ")
        res[-1].append(mtr[0])
        if (mtr[2] == "м"):
            res[-1].append(mtr[1])
        else:
            mtr[1] = mtr[1].replace(",", ".")
            mtr[1] = int(float(mtr[1]) * 1000)
            res[-1].append(mtr[1])
        for ad in elem.find_all("span", {"class": "e-tile-address"}):
            res[-1].append(ad.span.text)
f = open('domofond.txt', 'w')
for i in range (0, len(res)):
    for j in range (0, len(res[i])):
        f.write(str(res[i][j]).ljust(15))
    f.write('\n')
f.close()